In [21]:
#preprocessing
#model develop
#model_Train
#confusion_matrix, classificatrion report
#save the model
#load the model
#test the model with the new data

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
df = pd.read_csv(r'D:\ML_LAB\model_dev\datasets\Company_Data.csv')

In [24]:
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [25]:
df['Urban'].unique()

array(['Yes', 'No'], dtype=object)

In [26]:
df['US'].unique()

array(['Yes', 'No'], dtype=object)

In [27]:
df['Urban']=df['Urban'].replace('Yes',1)
df['Urban']=df['Urban'].replace('No',0)

df['US']=df['US'].replace('Yes',1)
df['US']=df['US'].replace('No',0)

In [28]:
from sklearn.preprocessing import LabelEncoder
lbl_enconder = LabelEncoder()

df['ShelveLoc']=lbl_enconder.fit_transform(df['ShelveLoc'])

In [29]:
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,1,1
1,11.22,111,48,16,260,83,1,65,10,1,1
2,10.06,113,35,10,269,80,2,59,12,1,1
3,7.40,117,100,4,466,97,2,55,14,1,1
4,4.15,141,64,3,340,128,0,38,13,1,0


In [30]:
df.shape

(400, 11)

In [31]:
# Random Sampling
US0=df[df['US']==0]
US1=df[df['US']==1]

print("US0: ", US0.shape)
print("US1: ", US1.shape)

US0:  (142, 11)
US1:  (258, 11)


In [32]:
no_sample=US1.sample(n=142)

In [33]:
no_sample.shape

(142, 11)

In [34]:
sampled_df=pd.concat([no_sample,US0],axis=0)

In [35]:
sampled_df.shape

(284, 11)

# HANDLING OUTLIERS

In [36]:
def outliers(df1, ft):
    Q1 = df1[ft].quantile(0.25)
    Q3 = df1[ft].quantile(0.75)
    IQR = Q3-Q1
    
    lb = Q1 - 1.5 * IQR
    ub = Q3 + 1.5 * IQR
    
    ls = df1.index[(df1[ft]<lb) | (df1[ft]>ub)]
    return ls

In [37]:
index_list = []
for feature in ['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'ShelveLoc', 'Age', 'Education']:
    index_list.extend(outliers(sampled_df,feature))

In [38]:
index_list

[25, 310, 42, 310, 165, 42, 125, 174, 313, 367]

In [39]:
def remove(df1, ls):
    ls = sorted(set(ls))
    df1 = df1.drop(ls)
    return df1

In [40]:
# df_new=df[~ index_list]

In [41]:
df_cleaned = remove(df,index_list)

In [42]:
df_cleaned.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,1,1
1,11.22,111,48,16,260,83,1,65,10,1,1
2,10.06,113,35,10,269,80,2,59,12,1,1
3,7.40,117,100,4,466,97,2,55,14,1,1
4,4.15,141,64,3,340,128,0,38,13,1,0


In [43]:
X = df_cleaned.drop('US',axis=1)
y = df_cleaned['US']

In [44]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X, y)
mutual_info

array([0.02976576, 0.        , 0.        , 0.38771274, 0.        ,
       0.00626688, 0.01482529, 0.01992541, 0.        , 0.        ])

In [45]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending=False)

Advertising    0.387713
Sales          0.029766
Age            0.019925
ShelveLoc      0.014825
Price          0.006267
CompPrice      0.000000
Income         0.000000
Population     0.000000
Education      0.000000
Urban          0.000000
dtype: float64

In [46]:
X = X.drop(['ShelveLoc','Population', 'CompPrice','Income'],axis=1)

In [47]:
X.head()

,Sales,Advertising,Price,Age,Education,Urban
0,9.50,11,120,42,17,1
1,11.22,16,83,65,10,1
2,10.06,10,80,59,12,1
3,7.40,4,97,55,14,1
4,4.15,3,128,38,13,1


In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [49]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(313, 6) (313,)
(79, 6) (79,)


In [50]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()

X_train_std=scaler.fit_transform(X_train)
X_test_std=scaler.transform(X_test)

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train_std, y_train).predict(X_test_std)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test_std.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 79 points : 7


In [52]:
gnb.score(X_train_std,y_train)

0.8690095846645367

In [53]:
gnb.score(X_test_std,y_test)

0.9113924050632911

In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.96      0.87        24
           1       0.98      0.89      0.93        55

    accuracy                           0.91        79
   macro avg       0.89      0.92      0.90        79
weighted avg       0.92      0.91      0.91        79



In [55]:
df['US'].value_counts()

US
1    258
0    142
Name: count, dtype: int64

# SAVING THE MODEL AND LOADING IT
TWO METHODS ->


1 - Pickle module


2 - Joblib module

In [56]:
#to save the model in the same directory
import pickle
pickle.dump(gnb, open("gnb_company_bayes_model.pkl", "wb"))

In [59]:
#loading the model which is saved
load_pickle_model = pickle.load(open("gnb_company_bayes_model_.pkl", "rb"))

NameError: name 'pickle' is not defined

In [58]:
import numpy as np
new_data = np.array([8.54, 0, 129, 42, 13, 1]) 
new_data_reshaped = new_data.reshape(1, 6)  #(no_of_rows, no_of_coloumns)
model_gnb = load_pickle_model.predict(new_data_reshaped)

predicted_label = model_gnb[0]  # Extracting the predicted label from the array
print("Predicted Label:", predicted_label)

NameError: name 'load_pickle_model' is not defined

In [ ]:
y_test.head()

148    1
286    1
71     1
380    1
335    1
Name: US, dtype: int64

In [ ]:
X_test.head()

,Sales,Advertising,Price,Age,Education,Urban
148,7.56,0,97,72,14,0
286,7.53,11,113,67,18,0
71,6.50,16,150,58,17,0
380,9.64,10,89,68,17,1
335,6.18,15,110,72,15,1
